## Generating artificial conversations for member call

In [7]:
import json
import pandas as pd
from openai import OpenAI
import requests
from tqdm import tqdm
import os
import random


with open('configuration.json') as f:
    data = json.load(f)
    
openai_base_url = data['openai_base_url']
openai_api_key = data['openai_api_key']

# Set the environmental variable for requests CA bundle and ssl cert file
os.environ["SSL_CERT_FILE"] = "root 1.pem"
os.environ["REQUESTS_CA_BUNDLE"] = "WellPoint_Internal_Root_CA 3.cer"

In [8]:


def get_completion(prompt: str, engine: str = "gpt-35-turbo-0613", system_prompt: str = None):
    messages = [{"role": "system", "content": system_prompt},
                {"role": "user", "content": prompt}]
    try:
        # Initialize the OpenAI client
        client = OpenAI(base_url=openai_base_url, api_key=openai_api_key)
        
        # Create completions
        response = client.chat.completions.create(
            model=engine,
            messages=messages,
            frequency_penalty=0,
            presence_penalty=0,
            temperature=0.9
        )
        
        response = json.loads(response)
        
        if response['responseCode'] != 200:
            print("response-----FAILED----------", response['responseDesc'])
            raise Exception("GPT gateway error")
        else:
            answer = response["choices"][0]["message"]["content"]
            return answer
    except Exception as e:
        print("ERROR---------", e)

In [9]:
test_data=[]

def generate_artificial_conversations(sample_data: str, num_conversations: int):
    # Define the system prompt
    system_prompt = """
    System Prompt:
    Your goal is to generate artificial conversations that closely resemble the real ones to facilitate testing and evaluation of our systems.    Each conversation should resemble a typical interaction between a provider and an agent in a healthcare insurance context.
    use this sample data as a reference {}
    
    Instructions:
    1. Pay attention to details such as language style, sentiment, and conversational flow.
    2. Aim to produce high-quality conversations that can be used for testing and analysis purposes.
    3. Do not repeat the same conversation transcript.

    
    """.format(sample_data)
    

    # Iterate over each data point and generate conversations
    ind=0
    topics=["Bills & Payments","Medical & Pharmacy Issues","Benefit & Coverage Inquiries","Provider Search","Claims & Authorizations","Appeals & Grievances","Technology Issues"]
    for i in tqdm(range(num_conversations)):
        prompt=f"""Please generate a hypothetical conversation where a member calls in with a query related to the topic: {topics[ind%7]}.    
            During the conversation, please ensure that the agent asks for the following information when appropriate:
                1. Full SSN or its last 4 digits
                2. Full Credit/Debit card info or its last 4 digits
                3. Credit Cards security code and expiry date
                4. Bank account number or its last 4 digits.
                5. Date of Birth
                6. Member ID if available
                7. Authorization codes and/or any other medical codes if applicable
            """
        ind+=1
        response=get_completion(prompt,system_prompt=system_prompt)
#         print("response---", response)
        test_data.append(response)
        
    print("res", test_data)

In [11]:
if __name__ == '__main__':
    sample_data = """
    Agent: Press one for Medicare Advantage Press two for Medicaid Press three for Medicare Supplement.
    Agent: Please use your telephone keypad to input. the three digit numeric plan code you are calling about now.
    Agent: Just a moment.
    Agent: All of our agents are.
    Agent: Thank you for calling paying plan customer service. This is John. How may I help you?
    Provider: Hey, this is Alice here with XYZ COMPANY. I'm actually doing a follow up to Renee. who called in about one of your members. We've been reaching out to a provider for her request That have been unsuccessful to see if we can get a, give an update to you guys.
    Agent: Okay, I'm sorry, I'm confused. so what is the update for?
    Provider: You guys member, there's an issue where your member is receiving a bill and it's being filed under a different individual and Rene called in to ask for us to reach out to this provider. We've tried contacting this provider as well sending a letter. So the next step for the claim Rep.
    Provider: Email asking me for tab. you guys reject the claim because we have not been successful and get in touch with the provider.
    Provider: I have a stuff number that you can reference.
    Agent: Okay.
    Agent: Hold on, just a second.
    Agent: And what's the stuff number?
    Provider: And I'm sorry, what did you say your name was?
    Agent: It's John.
    Provider: Okay, it is, 1234567890.
    Agent: And do you have the member three digit prefix and Id number?
    Provider: Yeah, I have the, ID number for.
    Provider: ABCDEFGH1234567890.
    Agent: And their name and date of birth.
    Provider: And she was v Brenda Spires with a date of birth of may 22 1990.
    Agent: Okay.
    Provider: The, the claim is coming on an Harris.
    Provider: Just the patient.
    Agent: Okay, those claims should have been denied.
    Agent: Yes.
    Agent: I can get that adjusted and denied.
    Provider: Okay.
    Agent: It like the system processed, so I don't know that's just weird that would do that. I mean you can clearly see, it's not the same.
    Provider: Yeah.
    Provider: Two five.
    Agent: Okay, so that's all you were wanting you were trying to update and wanted to.
    Provider: Yeah, that's how we were doing.
    Agent: Deny it on our end so that we can get this.
    Agent: Settled here but I can do that that's no problem.
    Agent: We say allows 7 to 14 business days but I'm gonna go ahead and take care of this.
    Provider: Okay.
    Agent: And I do have a reference number for you also.
    Provider: Alright back to that. What's your reference number?
    Agent: It's ABCD-1234-EFGH-5678.
    Provider: Alright, thank you so much.
    Agent: Thank you have a good day.
    Provider: You do the same, Thank you.
    Agent: Thank you. Bye, bye
    """
    generate_artificial_conversations(sample_data, num_conversations=100)



100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:09<00:00,  9.68s/it]

res ["Agent: Thank you for calling paying plan customer service. This is Sarah. How may I assist you today?\n\nMember: Hi Sarah, this is Mark. I have a question regarding my bills and payments.\n\nAgent: Of course, Mark. I'll be happy to help you. Before we proceed, may I have your full social security number or its last 4 digits for verification purposes?\n\nMember: Sure, it's 1234.\n\nAgent: Thank you, Mark. How can I assist you with your bills and payments?\n\nMember: I noticed that I've been charged twice for my last doctor's visit. The first charge went through, but the second one seems to be an error.\n\nAgent: I apologize for the inconvenience, Mark. To address this issue, I need your member ID if available, or your full name and date of birth, please.\n\nMember: My member ID is AB123456. \n\nAgent: Thank you, Mark. Let me pull up your account. I see the duplicate charge you're referring to. I will make sure to investigate and resolve this for you. Just to confirm, have you alre

In [ ]:
# print(test_data)